In [6]:
import mxnet as mx
import numpy as np
import cPickle


In [21]:
def make_def_pairs(data, labels, num_pairs, image_shape):
    pairs = np.zeros((num_pairs, 2, image_shape[0], image_shape[1],
                     image_shape[2])) # зачем двум картинкам 3 размера?
    labels_pairs = np.zeros(num_pairs)
    num_same = 0 
    num_dif = 0
    k = 0
    
    while k < num_pairs: # благодаря этой штуке мы вычлиняем из нашего batch любые две пары с их labels, верно? 
        first_index = np.random.randint(data.shape[0])
        second_index = np.random.randint(data.shape[0])
        same = labels[first_index] == labels[second_index]
        if same and num_same < num_pairs/2:
            pairs[k,0] = data[first_index]
            pairs[k,1] = data[second_index]
            labels_pairs[k] = 1
            num_same += 1
            k+=1
        elif not same and num_dif < num_pairs/2:
            pairs[k,0]=data[first_index]
            pairs[k,1]=data[second_index]
            labels_pairs[k]=0
            num_dif += 1
            k+=1
    return pairs, labels_pairs

class PairDataIter(mx.io.DataIter):
    def __init__(self, batch_size, mode='train'):
        super(PairDataIter).__init__() #зачем здесь множественное населдование? 
        assert mode in ['train', 'val'] # какая проверка условия здесь идёт? 
        self.batch_size = batch_size
        self.provide_data = [('data_a', (batch_size, 3, 32, 32)), 
                            ('data_b', (batch_size, 3, 32, 32))]
        # Здесь мог быть ваш итератор для cifar-10
        
    def extractImagesAndLabels(self, path, file):
        f=open(path+file, 'rb')
        dict=cPickle.load(f) # загрузка в словарь
        images=dict['data']
        images = np.reshape(images,(10000, 3,32, 32))
        labels = dict['labels']
        imagearray = mx.nd.array(images) # что эта шутка делает?
        labelarray = mx.nd.array(labels)
        return imagearray,labelarray
        
    def load_cifar(self, path):
        training_data=[]
        training_label=[]
        for f in ("data_batch_1", "data_batch_2", "data_batch_3", "data_batch_4", "data_batch_5"):
            imgarray, lblarray = extractImagesAndLabels(path, f)
            if not training_data:
                training_data = imgarray
                training_label = lblarray
            else:
                training_data = mx.nd.concatenate([training_data, imgarray])
                training_label = mx.nd.concatenate([training_label, lblarray])
        return training_data, training_label
    
    path="/home/alex/Документы/formxnet/cifar-10-batches-py/"
    
    training_data, training_label = load_cifar(path)
    
    valid_data, valid_label = extractImagesAndLabels(path, "test_batch")
    
    def next(self): # Здесь нужно сделать возвращение dataBatch, но чем они отличается от просто Batch? Гугл говорит, что это одно и то же
        pairs = make_pairs_random(self.data, self.label, self.batch_size, 
                                  (32, 32, 3))
        pass # зачем эта заглушка
        return mx.io.DataBatch(
            data=[mx.nd.array(np.moveaxis(pairs[0][:, 0], 3, 1)),
                 mx.nd.array(np.moveaxis(pairs[0][:, 1], 3, 1))],
            label = [mx.nd.array(pairs[1])],
            provide_data=self.provide_data,
            provide_label=self.provide_label        
            )
        

NameError: global name 'self' is not defined